In [4]:
from google.cloud import bigquery

/Users/oktavianu/miniforge3/envs/data-analysis/lib/python3.11/site-packages/google/cloud/bigquery/__init__.py:31: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution


In [5]:
# create clinet objt
client = bigquery.Client('intsql-2025')

/Users/oktavianu/miniforge3/envs/data-analysis/lib/python3.11/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [6]:
# datase ref to chicago crime
dataset_ref = client.dataset("chicago_crime", project="bigquery-public-data")

# API req, fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [7]:
# count table in this dataset
num_tables = list(client.list_tables(dataset))

print(len(num_tables))

1


In [8]:
# list all tables 
tables = list(client.list_tables(dataset))

# print tables
for table in tables:
    print(table.table_id)

crime


In [9]:
table_ref = dataset_ref.table("crime")

table = client.get_table(table_ref)

In [10]:
for field in table.schema:
    print(f"{field.name:20} | {field.field_type:10} | {field.description}")

unique_key           | INTEGER    | None
case_number          | STRING     | None
date                 | TIMESTAMP  | None
block                | STRING     | None
iucr                 | STRING     | None
primary_type         | STRING     | None
description          | STRING     | None
location_description | STRING     | None
arrest               | BOOLEAN    | None
domestic             | BOOLEAN    | None
beat                 | INTEGER    | None
district             | INTEGER    | None
ward                 | INTEGER    | None
community_area       | INTEGER    | None
fbi_code             | STRING     | None
x_coordinate         | FLOAT      | None
y_coordinate         | FLOAT      | None
year                 | INTEGER    | None
updated_on           | TIMESTAMP  | None
latitude             | FLOAT      | None
longitude            | FLOAT      | None
location             | STRING     | None


In [ ]:
client.list_rows(table, selected_fields=['longitude', 'latitude', 'x_coordinate', 'y_coordinate'], max_results=10).to_dataframe()

AttributeError: 'str' object has no attribute 'name'

In [16]:
field_names = [field.name for field in table.schema]
print("Available fields:", field_names)

Available fields: ['unique_key', 'case_number', 'date', 'block', 'iucr', 'primary_type', 'description', 'location_description', 'arrest', 'domestic', 'beat', 'district', 'ward', 'community_area', 'fbi_code', 'x_coordinate', 'y_coordinate', 'year', 'updated_on', 'latitude', 'longitude', 'location']


In [20]:
client.list_rows(
    table,
    selected_fields=table.schema[:10],
    max_results=10
).to_dataframe()

/var/folders/7m/y8yqbwcj4_s9pbq64p2cdkw00000gn/T/ipykernel_1282/3553454448.py:5: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  ).to_dataframe()


,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic
0,12461437,JE346938,2021-08-23 06:00:00+00:00,0000X E WACKER DR,0281,CRIMINAL SEXUAL ASSAULT,NON-AGGRAVATED,HOTEL / MOTEL,False,False
1,13395416,JH184337,2024-03-11 23:43:00+00:00,001XX N STATE ST,0281,CRIMINAL SEXUAL ASSAULT,NON-AGGRAVATED,CTA TRAIN,False,False
2,11419518,JB400424,2018-08-19 18:20:00+00:00,0000X E RANDOLPH ST,0312,ROBBERY,ARMED:KNIFE/CUTTING INSTRUMENT,STREET,True,False
3,2088091,HH313339,2002-04-17 14:40:31+00:00,001XX N CANAL ST,0313,ROBBERY,ARMED: OTHER DANGEROUS WEAPON,STREET,False,False
4,1794402,G619731,2001-10-15 11:15:00+00:00,0000X N CANAL ST,031A,ROBBERY,ARMED: HANDGUN,STREET,False,False
5,13491178,JH301061,2024-06-10 22:20:00+00:00,0000X E LAKE ST,031A,ROBBERY,ARMED - HANDGUN,CONVENIENCE STORE,False,False
6,5540810,HN350150,2007-05-18 13:00:00+00:00,006XX W WASHINGTON BLVD,031A,ROBBERY,ARMED: HANDGUN,PARKING LOT/GARAGE(NON.RESID.),False,False
7,6030795,HP134355,2008-01-20 19:04:04+00:00,005XX W VAN BUREN ST,031A,ROBBERY,ARMED: HANDGUN,RESTAURANT,False,False
8,13033463,JG212994,2023-04-06 11:15:00+00:00,003XX N MICHIGAN AVE,0320,ROBBERY,STRONG ARM - NO WEAPON,RESTAURANT,False,True
9,13568140,JH393137,2024-08-18 16:33:00+00:00,001XX N STATE ST,0320,ROBBERY,STRONG ARM - NO WEAPON,DEPARTMENT STORE,False,False


In [21]:
geo_columns = ['longitude', 'latitude', 'x_coordinate', 'y_coordinate']

selected_columns = [
    field for field in table.schema
    if field.name in geo_columns
]

geog_columns = client.list_rows(
    table,
    selected_fields=selected_columns,
).to_dataframe()

In [22]:
print(geog_columns)

         x_coordinate  y_coordinate   latitude  longitude
0           1173132.0     1899349.0  41.879242 -87.639747
1           1176621.0     1902155.0  41.886864 -87.626852
2           1173190.0     1898749.0  41.877594 -87.639552
3           1173177.0     1899236.0  41.878931 -87.639586
4           1172180.0     1899351.0  41.879269 -87.643243
...               ...           ...        ...        ...
8430738     1150357.0     1908511.0  41.904858 -87.723135
8430739     1151073.0     1908137.0  41.903818 -87.720514
8430740     1149855.0     1909684.0  41.908087 -87.724948
8430741     1150926.0     1913025.0  41.917234 -87.720926
8430742     1152121.0     1910560.0  41.910446 -87.716601

[8430743 rows x 4 columns]
